This notebook is a step by step guide about how to train a deep neural network (DNN) in the DeepDeconv framework.

In [1]:
## Set up the sys.path in order to be able to import our modules
import os
import sys
module_path = os.path.abspath(os.path.join('../python'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('/data/alpha-transform'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.75 

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Using TensorFlow backend.


We load the class of the network. This class must inherit from the DeepNet superclass. The method build_model has to be redefined in the child class with the wanted architecture. In our work, we use the network defined in deconvnNet. 

In [3]:
from DeepDeconv.deepnetFCS.DeconvNet_custom import DeconvNet

/home/cben-ali/Programs/deep-deconv/python/shape_constraint/cadmos_lib.py:149: ComplexWarning: Casting complex values to real discards the imaginary part
  coeff[s] = trafo.adjoint_transform(temp, do_norm=False)


# 1) Alexis net : pre training for 1 epoch
This is the first attempt at finding gamma automatically. We first train the network for one epoch and the choose gamma so the MSE and shape loss respect a certain ratio (~1)

In [4]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.01
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_Target_window_iter2'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)




Instructions for updating:
Colocations handled automatically by placer.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeconvNet_custom.py:295: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`
  self.model = Model(input = [inputs,window,norm], outputs =output)


Tensor("loss/conv2d_8_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss/conv2d_8_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 32)   128         conv2d_1[0][0]                   
___________________________________________________

In [5]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'new_targets/'+'image-*-multihdu_newtar.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(data_directory+'Target_Window/'+'Target*')
win_files.sort()
print(win_files)

SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-00-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-01-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-02-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-03-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-04-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-05-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-06-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-07-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-08-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-09-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_eu

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

#dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
#                        model_file='', nb_img_per_file=10000, 
#                        validation_set_size=10000, noise_std=None, SNR=SNR, 
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
#                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Target_window_iter2.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Instructions for updating:
Use tf.cast instead.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:399: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5947/5947 [==============================] - 5699s 958ms/step - loss: 0.6473 - weighted_mean_squared_error: 0.4092 - weighted_shape_metric: 0.2381 - val_loss: 0.4408 - val_weighted_mean_squared_error: 0.0517 - val_weighted_shape_metric: 0.3892

Epoch 00001: val_loss improved from inf to 0.44083, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Target_window_iter2.hdf5
2019-08-27_15:06:34 - Epoch: 1, val_loss: 0.440833, val_weighted_mean_squared_error: 0.051670, val_weighted_shape_metric: 0.389164, loss: 0.647645, weighted_mean_squared_error: 0.409566, weighted_shape_metric: 0.238079
Epoch 2/20
5947/5947 [==============================] - 5752s 967ms/step - loss: 0.0881 - weighted_mean_squared_error: 0.0363 - weighted_shape_metric: 0.0518 - val_loss: 0.1643 - val_weighted_mean_squared_error: 0.0284 - val_weighted_shape_metric: 0.1358

Epoch 00002: val_loss improved from 0.44083 to 0.16426, saving model to ShapeNet2D_claire_sc4_layer4x5


Epoch 00015: val_loss did not improve from 0.05581
2019-08-28_13:28:16 - Epoch: 15, val_loss: 0.097906, val_weighted_mean_squared_error: 0.022699, val_weighted_shape_metric: 0.075208, loss: 0.038660, weighted_mean_squared_error: 0.022973, weighted_shape_metric: 0.015687
Epoch 16/20
5947/5947 [==============================] - 5750s 967ms/step - loss: 0.0356 - weighted_mean_squared_error: 0.0221 - weighted_shape_metric: 0.0135 - val_loss: 5.1812 - val_weighted_mean_squared_error: 5.0776 - val_weighted_shape_metric: 0.1036

Epoch 00016: val_loss did not improve from 0.05581
2019-08-28_15:04:05 - Epoch: 16, val_loss: 5.181186, val_weighted_mean_squared_error: 5.077581, val_weighted_shape_metric: 0.103605, loss: 0.035636, weighted_mean_squared_error: 0.022138, weighted_shape_metric: 0.013498
Epoch 17/20
5947/5947 [==============================] - 5748s 967ms/step - loss: 0.0344 - weighted_mean_squared_error: 0.0221 - weighted_shape_metric: 0.0122 - val_loss: 11.3590 - val_weighted_mean_s

In [14]:
dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Gaussian_window.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1,keep_best_loss=True, initial_epoch=12)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Gaussian_window.hdf5
Memory usage for the model + one batch (GB): 2.809000
Cannot have access to best parameters for monitor for checkpoint
Epoch 13/20


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:399: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=(array([[[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=12, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


3111/5947 [==============>...............] - ETA: 45:43 - loss: 0.0227 - weighted_mean_squared_error: 0.0227 - weighted_shearlet_metric: 6.3010e-08

KeyboardInterrupt: 

In [18]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=1
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet_retry'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)


loss (None, 96, 96)
temp (None, 1, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
loss (None, 96, 96)
temp (None, 1, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 96, 96, 32)   288         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_247 (BatchN (None, 96, 96, 32)   128         conv2d_49[0][0]                  
__________________________________________________________________________________________________
activation_247 (Activation)     (None, 96, 96, 32)   0           batch_normalizatio

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet_retry.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/20
5947/5947 [==============================] - 5997s 1s/step - loss: 0.3688 - weighted_mean_squared_error: 0.3651 - weighted_shearlet_metric: 0.0037 - val_loss: 0.0367 - val_weighted_mean_squared_error: 0.0367 - val_weighted_shearlet_metric: 1.2250e-05

Epoch 00001: val_loss improved from inf to 0.03669, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet_retry.hdf5
2019-08-02_15:46:03 - Epoch: 1, val_loss: 0.036690, val_weighted_mean_squared_error: 0.036678, val_weighted_shearlet_metric: 0.000012, loss: 0.369258, weighted_mean_squared_error: 0.365553, weighted_shearlet_metric: 0.003705
Epoch 2/20
5947/5947 [==============================] - 5951s 1s/step - loss: 0.0297 

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=20
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)


In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

In [12]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=1
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

loss (None, 96, 96)
temp (None, 1, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
loss (None, 96, 96)
temp (None, 1, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 96, 96, 32)   288         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_124 (BatchN (None, 96, 96, 32)   128         conv2d_25[0][0]                  
__________________________________________________________________________________________________
activation_124 (Activation)     (None, 96, 96, 32)   0           batch_normalizatio

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/20
5947/5947 [==============================] - 5938s 998ms/step - loss: 0.1939 - weighted_mean_squared_error: 0.1932 - weighted_shearlet_metric: 6.6304e-04 - val_loss: 0.0579 - val_weighted_mean_squared_error: 0.0578 - val_weighted_shearlet_metric: 1.1037e-04

Epoch 00001: val_loss improved from inf to 0.05787, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet.hdf5
2019-07-30_20:53:16 - Epoch: 1, val_loss: 0.057871, val_weighted_mean_squared_error: 0.057761, val_weighted_shearlet_metric: 0.000110, loss: 0.194088, weighted_mean_squared_error: 0.193424, weighted_shearlet_metric: 0.000664
Epoch 2/20
5947/5947 [==============================] - 6095s 1s/step - loss: 0.0304 - wei

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.5
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)


In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

# 2) Calculating the gammas

In [5]:
import keras.backend as K
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
from DeepDeconv.utils.batch_utils import get_batch_from_fits
import numpy as np
test_data, target_data,weights_data = get_batch_from_fits(gal_files[1], idx_list=np.arange(10000), SNR=[20,100],
                         noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
                         image_dim=96, image_per_row=100,
                         deconv_mode=deconv_mode,shape_constraint=True,win_filename=win_files[1],win_hdu=0,mom_hdu=1)



KeyboardInterrupt: 

In [ ]:
dnn = DeepNet(model_file='/home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_1epoch_preshape_auto_test_newtar_gfiles.hdf5',shape_constraint=True,gamma=0)
pred=dnn.predict(test_data,verbose=1)

dnn.model.inputs[1]=K.variable(test_data[1])
dnn.model.inputs[2]=K.variable(test_data[2])
dnn.gamma=1



In [9]:
from keras.engine.training_utils import weighted_masked_objective
import keras.losses as losses
weighted_loss2 = weighted_masked_objective(losses.get('mse'))
weighted_loss = weighted_masked_objective(dnn.shape_metric)
pred=K.variable(pred)
target_data=K.variable(target_data)
weights_data=K.variable(weights_data)



In [10]:
shap=K.get_value(weighted_loss(target_data,pred,weights_data,None))
meanse=K.get_value(weighted_loss2(target_data,pred,weights_data,None))
print(shap, meanse)

15.633449 0.031680536


In [11]:
gamma1=meanse/shap     #ratio(mse/shp)=1
gamma2=(2*meanse)/shap #ratio=0.5
gamma3=meanse/(2*shap) #ratio=2

In [12]:
gamma2

0.004052917092588914

# 3) Restart : ratio=1
The ratio of mse/shp is of 1 which means we'll be using gamma1

In [7]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_1epoch_preshape_auto_test.hdf5",
             shape_constraint=True,gamma=0.002) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

Loading model...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_1epoch_preshape_auto_test.hdf5
Renaming as...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_1epoch_preshape_auto_test
Tensor("loss_1/conv2d_8_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_1:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_1:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_1/conv2d_8_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
[<tf.Tensor 'input_1_2:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'window_1:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'norm_1:0' shape=(?, 6, 1, 1) dtype=float32>]
['input_1', 'window', 'norm']


In [8]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=19, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_with_mom_test_restart_ratio1_oldtar.hdf5',
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_with_mom_test_restart_ratio1_oldtar.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/19


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:363: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=19, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


5947/5947 [==============================] - 4211s 708ms/step - loss: 0.1464 - weighted_mean_squared_error: 0.1321 - weighted_shape_metric: 0.0143 - val_loss: 0.6049 - val_weighted_mean_squared_error: 0.5751 - val_weighted_shape_metric: 0.0298

Epoch 00001: val_loss improved from inf to 0.60489, saving model to ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_with_mom_test_restart_ratio1_oldtar.hdf5
2019-07-12_17:47:27 - Epoch: 1, val_loss: 0.604888, val_weighted_mean_squared_error: 0.575086, val_weighted_shape_metric: 0.029802, loss: 0.146392, weighted_mean_squared_error: 0.132091, weighted_shape_metric: 0.014302
Epoch 2/19
5947/5947 [==============================] - 4209s 708ms/step - loss: 0.1307 - weighted_mean_squared_error: 0.1245 - weighted_shape_metric: 0.0062 - val_loss: 0.1460 - val_weighted_mean_squared_error: 0.1291 - val_weighted_shape_metric: 0.0169

Epoch 00002: val_loss improved from 0.60489 to 0.14599, saving model to ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_rel


Epoch 00015: val_loss did not improve from 0.12687
2019-07-13_10:19:48 - Epoch: 15, val_loss: 0.133151, val_weighted_mean_squared_error: 0.122201, val_weighted_shape_metric: 0.010950, loss: 0.122487, weighted_mean_squared_error: 0.119533, weighted_shape_metric: 0.002954
Epoch 16/19
5947/5947 [==============================] - 4130s 694ms/step - loss: 0.1207 - weighted_mean_squared_error: 0.1183 - weighted_shape_metric: 0.0025 - val_loss: 0.1300 - val_weighted_mean_squared_error: 0.1178 - val_weighted_shape_metric: 0.0122

Epoch 00016: val_loss did not improve from 0.12687
2019-07-13_11:28:37 - Epoch: 16, val_loss: 0.129961, val_weighted_mean_squared_error: 0.117789, val_weighted_shape_metric: 0.012172, loss: 0.120733, weighted_mean_squared_error: 0.118270, weighted_shape_metric: 0.002462
Epoch 17/19
5947/5947 [==============================] - 4193s 705ms/step - loss: 0.1264 - weighted_mean_squared_error: 0.1240 - weighted_shape_metric: 0.0024 - val_loss: 0.1316 - val_weighted_mean_sq

In [9]:
del dnn2

# 4) Restart : ratio = 0.5

In [12]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu_newtar.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'Gaussian_Window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-00-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-01-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-02-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-03-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-04-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-05-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-06-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-07-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-08-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-09-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_eu

In [13]:
del dnn2

# 5) Restart : ratio = 2

In [14]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_1epoch_preshape_auto_test_newtar_gfiles.hdf5",
             shape_constraint=True,gamma=gamma2) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

NameError: name 'gamma2' is not defined

In [21]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=19, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_with_mom_test_restart_ratio2_newtar.hdf5',
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_with_mom_test_restart_ratio2_newtar.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/19
5947/5947 [==============================] - 4301s 723ms/step - loss: 1.0875 - weighted_mean_squared_error: 0.4164 - weighted_shape_metric: 0.6711 - val_loss: 0.6655 - val_weighted_mean_squared_error: 0.2411 - val_weighted_shape_metric: 0.4243

Epoch 00001: val_loss improved from inf to 0.66547, saving model to ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_with_mom_test_restart_ratio2_newtar.hdf5
2019-07-06_18:25:29 - Epoch: 1, val_loss: 0.665466, val_weighted_mean_squared_error: 0.241134, val_weighted_shape_metric: 0.424332, loss: 1.086189, weighted_mean_squared_error: 0.416079, weighted_shape_metric: 0.670110
Epoch 2/19
5947/5947 [==============================] - 4275s 719ms/step - loss: 1.0


Epoch 00015: val_loss did not improve from 0.61739
2019-07-07_11:18:50 - Epoch: 15, val_loss: 0.635741, val_weighted_mean_squared_error: 0.215550, val_weighted_shape_metric: 0.420192, loss: 1.165665, weighted_mean_squared_error: 0.412911, weighted_shape_metric: 0.752754
Epoch 16/19
5947/5947 [==============================] - 4241s 713ms/step - loss: 1.1684 - weighted_mean_squared_error: 0.4143 - weighted_shape_metric: 0.7541 - val_loss: 0.6278 - val_weighted_mean_squared_error: 0.2096 - val_weighted_shape_metric: 0.4181

Epoch 00016: val_loss did not improve from 0.61739
2019-07-07_12:29:31 - Epoch: 16, val_loss: 0.627768, val_weighted_mean_squared_error: 0.209649, val_weighted_shape_metric: 0.418119, loss: 1.168893, weighted_mean_squared_error: 0.414468, weighted_shape_metric: 0.754425
Epoch 17/19
5947/5947 [==============================] - 4293s 722ms/step - loss: 1.0990 - weighted_mean_squared_error: 0.4051 - weighted_shape_metric: 0.6939 - val_loss: 0.6259 - val_weighted_mean_sq

In [22]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.0001
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma0.0001_newtar'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)


Tensor("loss_5/conv2d_16_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_5:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_5:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_5/conv2d_16_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 96, 96, 32)   288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 96, 96, 32)   128         conv2d_9[0][0]                   
_________________________________________

In [23]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.0001_newtar.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:363: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5947/5947 [==============================] - 4184s 704ms/step - loss: 0.5994 - weighted_mean_squared_error: 0.5571 - weighted_shape_metric: 0.0423 - val_loss: 0.3982 - val_weighted_mean_squared_error: 0.3716 - val_weighted_shape_metric: 0.0266

Epoch 00001: val_loss improved from inf to 0.39822, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.0001_newtar.hdf5
2019-07-07_17:19:32 - Epoch: 1, val_loss: 0.398222, val_weighted_mean_squared_error: 0.371633, val_weighted_shape_metric: 0.026590, loss: 0.599638, weighted_mean_squared_error: 0.557318, weighted_shape_metric: 0.042319
Epoch 2/20
5947/5947 [==============================] - 4192s 705ms/step - loss: 0.4446 - weighted_mean_squared_error: 0.4055 - weighted_shape_metric: 0.0391 - val_loss: 0.2161 - val_weighted_mean_squared_error: 0.1919 - val_weighted_shape_metric: 0.0242

Epoch 00002: val_loss improved from 0.39822 to 0.21610, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_g

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5",
             shape_constraint=True,gamma=0.001) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.001_restart.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

# Appendix for GPU options

In [ ]:
## extra imports to set GPU options
#import tensorflow as tf
#from keras import backend as k

###################################
# TensorFlow wizardry
#config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
#config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.5 

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Same with multiple GPUs, Tensorflow will pre-allocate the whole memory of all the GPUs.
Use the following to prevent it (only when your station has several GPUs like the SAPPCN63):

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd